In [40]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [41]:
targhe = [
"FY293YC", "FY295YC", "FY298YC", "FY294YC", "FY296YC", "FV903SK", "FV904SK", "FV906SK", "FV907SK", "FV908SK", "FZ330SC", "FV913SK", "FV914SK", "FY402YC", "FY403YC", "ZB477AN", "ZB473AN", "ZB474AN", "ZB476AN", "ZB475AN", "ZB478AN", "ZB137AR", "ZB139AR", "ZB150AR", "ZB127AR", "ZB132AR", "ZB128AR", "ZB131AR", "ZB130AR", "FY400YC", "ZB135AR", "ZB136AR", "ZB134AR", "ZB373AN", "FY401YC", "CW363HC", "CW367HC", "FP698BP", "FP699BP", "CW365HC", "CW368HC", "CN433CA", "FV989FV", "FV990FV", "FV991FV", "FV995FV", "FV996FV", "FV997FV", "FV992FV", "FV985FV", "FV987FV", "FV988FV", "FV993FV", "FV994FV", "EG181YE", "FV986FV", "EN971TN", "FY299YC", 
]

In [65]:
df = pd.read_excel("../excels/eventi_manutenzioni_esterne (da fatture).xlsx",
                   sheet_name="Categorie", usecols="A:H")
df.head()

,ID,Targa,Data,Revisione,Tagliando,Categoria componente,Componente,Manutenzione
0,397,FV908SK,2021-05-12,No,No,Impianto di scarico,Filtro antiparticolato,Sostituzione
1,397,FV908SK,2021-05-12,No,No,Generale,Guarnizioni,Sostituzione
2,397,FV908SK,2021-05-12,No,No,Impianto di scarico,Dosatore riducente,Sostituzione
3,397,FV908SK,2021-05-12,No,No,Sensoristica,Taratura tachigrafo,NaN
4,398,FZ330SC,2021-05-13,No,No,Impianto di lubrificazione motore,Ingrassaggio,Pulizia


In [46]:
df.shape

(276, 8)

In [53]:
df = df[(df['Categoria componente']!='Generale') & (df['Categoria componente']!= '?')&(df.Tagliando == "No")&(df.Revisione == "No")]
df.shape

(208, 8)

In [54]:
df_gb = df.groupby(['Targa','Categoria componente']).size().reset_index().rename(columns={0:'count'})
df_gb

,Targa,Categoria componente,count
0,CW363HC,Impianto frenante,3
1,CW365HC,Impianto di lubrificazione motore,2
2,CW365HC,Impianto elettrico,2
3,CW365HC,Meccanica,1
4,EN971TN,Impianto frenante,1
...,...,...,...
108,ZB476AN,Impianto elettrico,1
109,ZB477AN,Impianto elettrico,4
110,ZB478AN,Idraulica,2
111,ZB478AN,Impianto di lubrificazione motore,1


In [55]:
df_tot = pd.DataFrame()

for i,j in zip(df_gb['Targa'], df_gb['Categoria componente']):
    df_sub = df[(df['Targa']==i) & (df['Categoria componente']==j)]
    df_sub = df_sub.reset_index(drop=True)
    df_sub['label'] = 0
    for i in range(df_sub.shape[0]-1):
        if(((df_sub['Data'][i+1] - df_sub['Data'][i]).days) < 7):
            df_sub.at[i+1, 'label'] = 1
            
    frames = [df_tot, df_sub]
    df_tot = pd.concat(frames)

In [59]:
df_tot#.to_csv("tmp.csv", index=False)

,ID,Targa,Data,Revisione,Tagliando,Categoria componente,Componente,Manutenzione,label
0,435,CW363HC,2021-07-10,No,No,Impianto frenante,Pastiglie freni,Sostituzione,0
1,435,CW363HC,2021-07-10,No,No,Impianto frenante,Pinze freni,Sostituzione,1
2,435,CW363HC,2021-07-10,No,No,Impianto frenante,Liquido freni,Sostituzione,1
0,469,CW365HC,2021-09-10,No,No,Impianto di lubrificazione motore,Pompa,Sostituzione,0
1,469,CW365HC,2021-09-10,No,No,Impianto di lubrificazione motore,Filtro gasolio,Sostituzione,1
...,...,...,...,...,...,...,...,...,...
0,401,ZB478AN,2021-05-17,No,No,Idraulica,Valvola distributore,Sostituzione,0
1,439,ZB478AN,2021-07-24,No,No,Idraulica,Valvola pistone,Sostituzione,0
0,401,ZB478AN,2021-05-17,No,No,Impianto di lubrificazione motore,Rabbocco olio idraulico,Sostituzione,0
0,401,ZB478AN,2021-05-17,No,No,Meccanica,Pistone vasca,Sostituzione,0


In [57]:
df_tot['label'].value_counts()

0    149
1     58
Name: label, dtype: int64

In [60]:
df_tot = df_tot[df_tot['label']==0]
df_tot = df_tot.reset_index(drop=True)
# df_tot.to_excel("fatture_filtered.xlsx", index=False)

In [61]:
df_tot

,ID,Targa,Data,Revisione,Tagliando,Categoria componente,Componente,Manutenzione,label
0,435,CW363HC,2021-07-10,No,No,Impianto frenante,Pastiglie freni,Sostituzione,0
1,469,CW365HC,2021-09-10,No,No,Impianto di lubrificazione motore,Pompa,Sostituzione,0
2,469,CW365HC,2021-09-10,No,No,Impianto elettrico,Fanali,Sostituzione,0
3,469,CW365HC,2021-09-10,No,No,Meccanica,Balestre,Sostituzione,0
4,416,EN971TN,2021-06-15,No,No,Impianto frenante,Kit Frizione,Sostituzione,0
...,...,...,...,...,...,...,...,...,...
144,401,ZB478AN,2021-05-17,No,No,Idraulica,Valvola distributore,Sostituzione,0
145,439,ZB478AN,2021-07-24,No,No,Idraulica,Valvola pistone,Sostituzione,0
146,401,ZB478AN,2021-05-17,No,No,Impianto di lubrificazione motore,Rabbocco olio idraulico,Sostituzione,0
147,401,ZB478AN,2021-05-17,No,No,Meccanica,Pistone vasca,Sostituzione,0
